# Collecting Tweets using Tweepy

In [26]:
# Load packages
import tweepy as tw #make sure to install v3.10
import pandas as pd
import numpy as np
# install preprocessor using pip install via Command Prompt/Anaconda prompt: pip install tweet-preprocessor
from preprocessor import api as p
#from preprocessor.api import clean, tokenize, parse #to clean tweets

In [2]:
# Credentials
access_token = "1311658729201377281-alEuRquK89jGbga7ckTsFhIlkRVPOY"
access_token_secret = "rpQowQaio69m3XMByxXEzUdo73WPnpaPuYTQpAFH9cjoa"
api_key = "E0RK2oHynrMIJ9YVrvuhIIjc6"
api_key_secret = "kHnfmtYn7J66MkR7bUE3BTnWsW64ZUDUIxtw9bThP5kmDjq0Vl"

In [3]:
# Authentication app
auth = tw.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

In [113]:
# Find by phrase or keywords within a specified time
search_words = '"utah lake" has:links'
#search_words = '"utah lake" -has:links' 
# we can not use -filter:links etc for premium API search
# use this link instead https://developer.twitter.com/en/docs/twitter-api/premium/rules-and-filtering/using-premium-operators
# it is equivalent to utah lake until:2020-12-20 since:2020-11-28 -filter:links
# we can't filter out retweets in the premium search

# advanced search in Twitter search box 
# utah lake (#utahlake) until:2020-12-03 since:2020-11-28 -filter:links -filter:retweets
# If I include hashtag in search, it wouldn't show the ones without hashtag
# May be we can have a separate search by #utahlake and then combine both the dataframes

# From date to date, for premium API search the date should be in format yyyyMMddHHmm
year = int(input("Year: "))
date_since = f'{year}01010000' #from January 1st, 12:00AM UTC
date_until = f'{year}12312359' #to December 31st, 11:59PM UTC
#date_until = f'{year}01010000' # for 2020

# Create a cursor object that iterates over the requested information
# had to create a dev environment named "SentimentAnalysis" in Twitter's Developer Portal
tweets = tw.Cursor(api.search_full_archive, environment_name = "SentimentAnalysis",
                   query=search_words,
                   fromDate=date_since, toDate=date_until).items()

#when we have premium subscription we will use this
#API.search_full_archive(environment_name, query[, tag][, fromDate][, toDate][, maxResults][, next])

# Parse the cursor object into rows of lists using list comprehension
rows = [[tweet.user.location, tweet.text, tweet.created_at, tweet.coordinates] for tweet in tweets]
# if required, look into user.derived.locations.geo
# it assigns lat/long based on user's profile and the geographic precision they set
# for example, if a user set their precision at the country-level, it will show center of US at lat/long
# so it is not too accurate and might be irrelevant to us
# https://developer.twitter.com/en/docs/twitter-api/enterprise/enrichments/overview/profile-geo

Year: 2019


In [114]:
# Create dataframe from the rows of lists
df_tweet = pd.DataFrame(data=rows, columns=['location', 'tweet', 'time', 'coordinates'])

In [115]:
# Add ID column (because we removed user information)
unique_id = np.arange(len(df_tweet))
df_tweet['id'] = unique_id + 1
df_tweet = df_tweet[['id', 'location', 'tweet', 'time', 'coordinates']]

In [116]:
df_tweet.head(50)

,id,location,tweet,time,coordinates
0,1,"Utah, USA",@WeDontEnd Come fish a Utah lake with me broth...,2019-12-31 20:55:57,None
1,2,"Indian Hills, Provo Utah",The lights are still up at the marina😊 @ Utah ...,2019-12-31 02:33:25,None
2,3,"Provo, UT",Want to learn more about Utah Lake?\nOur blog ...,2019-12-30 16:15:09,None
3,4,"Путин, забери домой дурака!",Utah Lake [1375x1080] [OC] (Original posted to...,2019-12-28 04:18:14,None
4,5,"Путин, забери домой дурака!","Utah Lake, Dec. 19, 2019 [1580x1052] [OC] (Ori...",2019-12-27 09:32:29,None
5,6,"Путин, забери домой дурака!","Utah Lake, Dec, 19, 2019 [1620x1080] [OC] (Ori...",2019-12-27 02:02:14,None
6,7,"Provo, UT",RT @TheAvgJoeRun: 10k Night Run by Utah Lake i...,2019-12-24 04:22:57,None
7,8,None,10k Night Run by Utah Lake in Saratoga Springs...,2019-12-24 03:23:16,None
8,9,"Atlanta, GA",RT @CryptidQuest: Prior to the arrival of west...,2019-12-23 22:43:41,None
9,10,None,"Prior to the arrival of western settlers, nati...",2019-12-23 20:16:42,None


In [117]:
df_tweet.tail()

,id,location,tweet,time,coordinates
1786,1787,None,RT @EarthCaptres: Storm over Utah Lake ⛈ https...,2019-01-01 15:35:32,None
1787,1788,None,RT @EarthCaptres: Storm over Utah Lake ⛈ https...,2019-01-01 15:11:14,None
1788,1789,st george,Can we do this to Utah lake https://t.co/0Ud06...,2019-01-01 13:05:39,None
1789,1790,None,RT @EarthCaptres: Storm over Utah Lake ⛈ https...,2019-01-01 11:32:03,None
1790,1791,"Ampuriabrava, España 🇪🇸",RT @EarthCaptres: Storm over Utah Lake ⛈ https...,2019-01-01 11:08:23,None


In [118]:
df_tweet.tweet[1]

'The lights are still up at the marina😊 @ Utah Lake State Park https://t.co/3OTCez6Jm9'

# Cleaning tweets in the dataframe 

In [119]:
# Clean each tweet in our dataframe using the preprocessor module
tweet_clean = []
for tweet in df_tweet['tweet']:
    cleaned = p.clean(tweet)
    tweet_clean.append(cleaned)

In [120]:
# Store as a new dataframe
df_tweet_clean = df_tweet.copy()
df_tweet_clean['tweet'] = tweet_clean

In [121]:
df_tweet_clean.head()

,id,location,tweet,time,coordinates
0,1,"Utah, USA",Come fish a Utah lake with me brother,2019-12-31 20:55:57,None
1,2,"Indian Hills, Provo Utah",The lights are still up at the marina Utah Lak...,2019-12-31 02:33:25,None
2,3,"Provo, UT",Want to learn more about Utah Lake?Our blog is...,2019-12-30 16:15:09,None
3,4,"Путин, забери домой дурака!",Utah Lake [1375x1080] [OC] (Original posted to...,2019-12-28 04:18:14,None
4,5,"Путин, забери домой дурака!","Utah Lake, Dec. , [1580x1052] [OC] (Original p...",2019-12-27 09:32:29,None


# Save the dataframe as a CSV

In [122]:
# Checking current working directory
import os
os.getcwd()

'C:\\Users\\rapiduser\\Documents\\WaterQualityPerception\\Pierre\\WaterQualityPerceptionMP2020'

In [123]:
# Store as CSV to a relative path
df_tweet_clean.to_csv(f"..\\..\\Data\\tweets_{year}_links.csv")
#df_tweet_clean.to_csv("..\\..\\Data\\scraped_tweets_test.csv")